# Goals:
- 
- 

In [34]:
#Importing Required Libraries
#Data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Model 
from sklearn import linear_model
import requests

## Datasets



Our Dataset source: https://www.energidataservice.dk/tso-electricity/ConsumptionPerGridarea


In [35]:

# The API endpoint
url = "https://api.energidataservice.dk/dataset/ConsumptionPerGridarea/download?start=2023-12-01&end=2024-01-01&limit=0"

# A GET request to the API
response = requests.get(url)
result = response.json()

In [36]:
df = pd.DataFrame(result)
df

,HourUTC,HourDK,GridCompany,ResidualConsumption,FlexSettledConsumption,HourlySettledConsumption
0,2023-12-31T22:00:00,2023-12-31T23:00:00,003,None,92941.889,0.000
1,2023-12-31T22:00:00,2023-12-31T23:00:00,007,None,68369.987,0.000
2,2023-12-31T22:00:00,2023-12-31T23:00:00,016,None,461.403,30.360
3,2023-12-31T22:00:00,2023-12-31T23:00:00,031,None,67108.002,89261.926
4,2023-12-31T22:00:00,2023-12-31T23:00:00,042,None,10069.721,8485.240
...,...,...,...,...,...,...
37939,2023-11-30T23:00:00,2023-12-01T00:00:00,953,None,17630.500,NaN
37940,2023-11-30T23:00:00,2023-12-01T00:00:00,954,None,2715.000,NaN
37941,2023-11-30T23:00:00,2023-12-01T00:00:00,960,None,492.800,NaN
37942,2023-11-30T23:00:00,2023-12-01T00:00:00,962,None,0.000,NaN


In [37]:
# Saving the data frame in CSV format, so we would have our local dataset. 
df.to_csv('ConsumptionPerGridarea20241201.csv', index=False)

## Data Exploration

In [38]:
df_loc = pd.read_csv('ConsumptionPerGridarea20241201.csv')  

In [39]:
df_loc.head()

,HourUTC,HourDK,GridCompany,ResidualConsumption,FlexSettledConsumption,HourlySettledConsumption
0,2023-12-31T22:00:00,2023-12-31T23:00:00,3,NaN,92941.889,0.000
1,2023-12-31T22:00:00,2023-12-31T23:00:00,7,NaN,68369.987,0.000
2,2023-12-31T22:00:00,2023-12-31T23:00:00,16,NaN,461.403,30.360
3,2023-12-31T22:00:00,2023-12-31T23:00:00,31,NaN,67108.002,89261.926
4,2023-12-31T22:00:00,2023-12-31T23:00:00,42,NaN,10069.721,8485.240


In [40]:
print(df_loc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37944 entries, 0 to 37943
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   HourUTC                   37944 non-null  object 
 1   HourDK                    37944 non-null  object 
 2   GridCompany               37944 non-null  int64  
 3   ResidualConsumption       0 non-null      float64
 4   FlexSettledConsumption    37944 non-null  float64
 5   HourlySettledConsumption  31248 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.7+ MB
None


In [41]:
# the HourUTC and ResidualConsumption are removed, since we will not use them 
df_loc = df_loc.drop(['HourUTC', 'ResidualConsumption'], axis=1)

In [42]:
#change codes to categories
df_loc['GridCompany'] = df_loc['GridCompany'].astype(str)

In [43]:
df_loc.head()

,HourDK,GridCompany,FlexSettledConsumption,HourlySettledConsumption
0,2023-12-31T23:00:00,3,92941.889,0.000
1,2023-12-31T23:00:00,7,68369.987,0.000
2,2023-12-31T23:00:00,16,461.403,30.360
3,2023-12-31T23:00:00,31,67108.002,89261.926
4,2023-12-31T23:00:00,42,10069.721,8485.240


In [44]:
##changing the company number to a category format:
df_loc['GridCompany'] = df_loc['GridCompany'].astype('category')

In [45]:
print(df_loc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37944 entries, 0 to 37943
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   HourDK                    37944 non-null  object  
 1   GridCompany               37944 non-null  category
 2   FlexSettledConsumption    37944 non-null  float64 
 3   HourlySettledConsumption  31248 non-null  float64 
dtypes: category(1), float64(2), object(1)
memory usage: 928.9+ KB
None


In [46]:
df_loc.shape

(37944, 4)

In [47]:
df_loc.describe()

,FlexSettledConsumption,HourlySettledConsumption
count,37944.000000,31248.000000
mean,48943.723912,59510.880285
std,108601.314891,115928.576942
min,0.000000,0.000000
25%,1937.268250,2099.929250
50%,9546.420000,8833.887000
75%,40588.081000,53184.184250
max,875322.393000,714327.223000


In [48]:
#unique company codes number
com_num = df_loc['GridCompany'].nunique()
com_num

51

## Company names data set

In [73]:

# The API endpoint
url_comp = "https://api.energidataservice.dk/dataset/PowerSupplierChangePerGridarea/download?start=2023-12-01&end=2024-01-01&limit=0"

# A GET request to the API
response_comp = requests.get(url_comp)
result_comp = response_comp.json()

In [74]:
df_comp = pd.DataFrame(result_comp)
df_comp.head()

,Month,GridCompany,GridCompanyName,NumberofChangesforSmallConsumers,NumberofChangesforLargeConsumers
0,2023-12-01T00:00:00,016,El-net Kongerslev A/S,22,0
1,2023-12-01T00:00:00,031,Nord Energi Net A/S,1501,20
2,2023-12-01T00:00:00,042,Netselskabet Elværk A/S - 042,127,4
3,2023-12-01T00:00:00,051,Elinord A/S,336,1
4,2023-12-01T00:00:00,084,Kimbrer Elnet A/S,70,0


In [78]:
# Saving the data frame in CSV format, so we would have our local dataset. 
df_comp.to_csv('PowerSupplierChangePerGridarea.csv', index=False)

In [79]:
df_comp = pd.read_csv('PowerSupplierChangePerGridarea.csv')  

In [92]:
df_comp.head()

,Month,GridCompany,GridCompanyName,NumberofChangesforSmallConsumers,NumberofChangesforLargeConsumers
0,2023-12-01T00:00:00,16,El-net Kongerslev A/S,22,0
1,2023-12-01T00:00:00,31,Nord Energi Net A/S,1501,20
2,2023-12-01T00:00:00,42,Netselskabet Elværk A/S - 042,127,4
3,2023-12-01T00:00:00,51,Elinord A/S,336,1
4,2023-12-01T00:00:00,84,Kimbrer Elnet A/S,70,0


In [93]:
#change codes to categories
df_comp['GridCompany'] = df_comp['GridCompany'].astype(str)
df_comp['GridCompany'] = df_comp['GridCompany'].astype('category')

In [94]:
df_comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Month                             40 non-null     object  
 1   GridCompany                       40 non-null     category
 2   GridCompanyName                   40 non-null     object  
 3   NumberofChangesforSmallConsumers  40 non-null     int64   
 4   NumberofChangesforLargeConsumers  40 non-null     int64   
dtypes: category(1), int64(2), object(2)
memory usage: 2.8+ KB


In [95]:
#unique company codes number
com_num_c = df_comp['GridCompany'].nunique()
com_num_c

40

In [96]:
GridCompany = df_comp['GridCompanyName'].unique()
GridCompany

array(['El-net Kongerslev A/S', 'Nord Energi Net A/S',
       'Netselskabet Elværk A/S - 042', 'Elinord A/S',
       'Kimbrer Elnet A/S', 'Læsø Elnet A/S', 'N1 A/S - 131',
       'Hammel Elforsyning Net A/S', 'Konstant Net A/S - 151',
       'Elnet Midt A/S', 'Dinel A/S - 233', 'TREFOR El-Net A/S',
       'Konstant Net A/S - 245', 'Netselskabet Elværk A/S - 331',
       'Grindsted Elnet A/S', 'Ikast El Net A/S', 'N1 A/S - 344',
       'NOE Net A/S', 'RAH Net A/S', 'L-Net A/S', 'Forsyning Elnet A/S',
       'Aal El-net A.m.b.a', 'Hjerting Transformatorforening',
       'Hurup Elværk Net A/S', 'Tarm Elværk Net A/S', 'Videbæk Elnet A/S',
       'Sunds Net A.m.b.a', 'N1 A/S - 398', 'Ravdex A/S', 'Veksel A/S',
       'FLOW Elnet A/S', 'Vores Elnet A/S',
       'Midtfyns Elforsyning A.m.b.A.', 'Cerius A/S - 740',
       'Elektrus A/S', 'Radius Elnet A/S', 'Cerius A/S - 853',
       'NKE-Elnet A/S', 'Zeanet A/S', 'TREFOR El-Net Øst A/S'],
      dtype=object)

In [97]:
#unique code categories in name dataset
df_comp['GridCompany'].unique()

['16', '31', '42', '51', '84', ..., '791', '853', '854', '860', '911']
Length: 40
Categories (40, object): ['131', '141', '151', '154', ..., '853', '854', '860', '911']

In [98]:
#unique code categories in energy dataset
df_loc['GridCompany'].unique()

['3', '7', '16', '31', '42', ..., '953', '954', '960', '962', '990']
Length: 51
Categories (51, object): ['131', '141', '151', '154', ..., '954', '960', '962', '990']

In [105]:
#Combine the company names with energy data
#df_join = df_loc.join(df_comp.set_index('GridCompany'), on='GridCompany')
# inner join
df_join = pd.merge(df_loc, df_comp, on='GridCompany', how='inner')
df_join = df_join.drop(['HourDK'], axis=1)

In [106]:
df_join.head()

,GridCompany,FlexSettledConsumption,HourlySettledConsumption,Month,GridCompanyName,NumberofChangesforSmallConsumers,NumberofChangesforLargeConsumers
0,16,461.403,30.360,2023-12-01T00:00:00,El-net Kongerslev A/S,22,0
1,31,67108.002,89261.926,2023-12-01T00:00:00,Nord Energi Net A/S,1501,20
2,42,10069.721,8485.240,2023-12-01T00:00:00,Netselskabet Elværk A/S - 042,127,4
3,51,11408.555,20658.430,2023-12-01T00:00:00,Elinord A/S,336,1
4,84,1828.330,602.030,2023-12-01T00:00:00,Kimbrer Elnet A/S,70,0


In [108]:
#The sum of consumption for december 2023
df_sum = df_join.groupby("GridCompany").sum()
df_sum

,FlexSettledConsumption,HourlySettledConsumption,Month,GridCompanyName,NumberofChangesforSmallConsumers,NumberofChangesforLargeConsumers
GridCompany,,,,,,
131,2.529579e+08,2.782043e+08,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,N1 A/S - 131N1 A/S - 131N1 A/S - 131N1 A/S - 1...,6104520,34968
141,1.559040e+06,1.515751e+06,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Hammel Elforsyning Net A/SHammel Elforsyning N...,49848,0
151,7.725873e+07,1.206301e+08,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Konstant Net A/S - 151Konstant Net A/S - 151Ko...,1894224,8928
154,9.622067e+06,2.354859e+07,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Elnet Midt A/SElnet Midt A/SElnet Midt A/SElne...,203112,3720
16,4.085362e+05,4.494674e+04,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,El-net Kongerslev A/SEl-net Kongerslev A/SEl-n...,16368,0
233,5.576922e+07,3.978753e+07,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Dinel A/S - 233Dinel A/S - 233Dinel A/S - 233D...,1742448,13392
244,6.949458e+07,9.916966e+07,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,TREFOR El-Net A/STREFOR El-Net A/STREFOR El-Ne...,1796760,23064
245,3.402493e+07,4.359019e+07,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Konstant Net A/S - 245Konstant Net A/S - 245Ko...,1023000,14136
31,5.284470e+07,6.213133e+07,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Nord Energi Net A/SNord Energi Net A/SNord Ene...,1116744,14880


In [112]:
#Top 10 Flex Companies 
top_flex_consumers = df_sum.sort_values('FlexSettledConsumption', ascending=False).head(10)
top_flex_consumers

,FlexSettledConsumption,HourlySettledConsumption,Month,GridCompanyName,NumberofChangesforSmallConsumers,NumberofChangesforLargeConsumers
GridCompany,,,,,,
791,4.494429e+08,3.945404e+08,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Radius Elnet A/SRadius Elnet A/SRadius Elnet A...,16896984,340752
131,2.529579e+08,2.782043e+08,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,N1 A/S - 131N1 A/S - 131N1 A/S - 131N1 A/S - 1...,6104520,34968
740,2.278401e+08,1.653560e+08,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Cerius A/S - 740Cerius A/S - 740Cerius A/S - 7...,4006440,130200
344,1.908204e+08,2.294916e+08,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,N1 A/S - 344N1 A/S - 344N1 A/S - 344N1 A/S - 3...,3566736,51336
543,8.952542e+07,1.655501e+08,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Vores Elnet A/SVores Elnet A/SVores Elnet A/SV...,1857024,28272
151,7.725873e+07,1.206301e+08,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Konstant Net A/S - 151Konstant Net A/S - 151Ko...,1894224,8928
244,6.949458e+07,9.916966e+07,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,TREFOR El-Net A/STREFOR El-Net A/STREFOR El-Ne...,1796760,23064
233,5.576922e+07,3.978753e+07,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Dinel A/S - 233Dinel A/S - 233Dinel A/S - 233D...,1742448,13392
31,5.284470e+07,6.213133e+07,2023-12-01T00:00:002023-12-01T00:00:002023-12-...,Nord Energi Net A/SNord Energi Net A/SNord Ene...,1116744,14880
